In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cmath

from astropy.io import fits
from astropy.table import Table

## 1. get data -- low-resolution metrics dataset

In [6]:
input_path = f"../data/test"  ### 20000 test stars
dat = np.load(f"{input_path}/test-metrics.npy", allow_pickle=True)
metrics_dataset = dat[()]

2024-09-02 16:25:50.275021: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 16:25:50.347202: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 16:25:50.416089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 16:25:50.476497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 16:25:50.494834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 16:25:50.608495: I tensorflow/core/platform/cpu_feature_gu

In [7]:
metrics_dataset['PSFs']

{'psf_GT': <tf.Tensor: shape=(20000, 32, 32), dtype=float32, numpy=
 array([[[1.79118233e-05, 3.12717930e-05, 4.55787704e-05, ...,
          5.42988528e-06, 5.47128093e-06, 5.79764355e-06],
         [1.05958643e-05, 2.07999001e-05, 4.02448677e-05, ...,
          6.28061935e-06, 6.89872513e-06, 6.92308595e-06],
         [6.97893029e-06, 1.39370713e-05, 2.94485671e-05, ...,
          8.01858459e-06, 8.25699226e-06, 7.44693807e-06],
         ...,
         [5.12935503e-06, 7.24337906e-06, 8.99597580e-06, ...,
          1.51253780e-05, 9.64287847e-06, 6.12996018e-06],
         [6.59970647e-06, 7.95434971e-06, 8.62416164e-06, ...,
          1.43891384e-05, 1.29744467e-05, 7.28984742e-06],
         [6.61288914e-06, 7.69221424e-06, 8.11100290e-06, ...,
          1.24654689e-05, 1.33101621e-05, 1.06677626e-05]],
 
        [[9.98072846e-06, 1.91733834e-05, 3.22902124e-05, ...,
          7.73896681e-06, 6.68459597e-06, 5.07290133e-06],
         [6.22386415e-06, 1.07690612e-05, 2.25511267e-05, ...

In [8]:
psf_Data = metrics_dataset['PSFs']

## 2. to DataFrame with model keys

In [9]:
df = pd.DataFrame()

In [10]:
df['RA'] = psf_Data['position'][psf_Data['Flag']==1][:,0]*0.5/1000
df['Dec'] = psf_Data['position'][psf_Data['Flag']==1][:,1]*0.5/1000
df['E1_STAR'] = psf_Data['GT_pred_e1_HSM']
df['E2_STAR'] = psf_Data['GT_ped_e2_HSM']
df['E1_PSF'] = psf_Data['pred_e1_HSM']
df['E2_PSF'] = psf_Data['pred_e2_HSM']
df['FLAG_STAR'] = 1
df['FLAG_PSF'] = 1
df['SIGMA_STAR'] = psf_Data['GT_pred_R2_HSM']
df['SIGMA_PSF'] = psf_Data['pred_R2_HSM']

## Save to .fits file

In [11]:
t = Table.from_pandas(df)
t.write(f"{input_path}/Euclid_rho_input.fits", overwrite=True)

## Test to read the .fits file

In [13]:
test_rd = fits.open(f"{input_path}/Euclid_rho_input.fits", memmap=False)

In [14]:
np.shape(test_rd[1].data['RA'])

(7429,)